In [1]:
##https://www.statology.org/matplotlib-rectangle/
from PIL import Image
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt
import girder_client
import dsaSecrets 
import random
from matplotlib.patches import Rectangle
from IPython.display import display
import matplotlib.pylab as plt
import numpy as np
from ipywidgets import interact,FloatSlider,IntSlider,interactive,HBox,VBox, fixed, Button
import ipywidgets as widgets
from IPython.display import display

from IPython.display import clear_output


%matplotlib inline

%load_ext autoreload
%autoreload 2
import tmaHelpers as tma

gc = girder_client.GirderClient(apiUrl=dsaSecrets.apiUrl)
gc.authenticate(apiKey=dsaSecrets.apiToken)
## Samples Images 62bb03e4e70def4ae72689f5

%matplotlib inline

In [2]:
## SEt all the variables needed to pull an image, as well as the fiducial points.. for now the fiducials are at 0.625X
studyImageId = '62ded6cbe70def4ae726b228'
studyImages = list(gc.listItem(studyImageId))

thumbImageCache = {} ## Create a local in memory cache for downloaded thumbs
dispMag = 0.625 ### For testing I don't want to try and view the entire 20 or 40X image..

imgId = studyImages[0]['_id']
imgThumb = gc.get("item/%s/tiles/region?units=base_pixel&magnification=%s" % (imgId, dispMag),jsonResp=False)
region_im = np.array(Image.open(BytesIO(imgThumb.content)))
# imgId = '62bb03e4e70def4ae72689f5'

# imgId='62ded6e6e70def4ae726b3fd'

# imgId = '62ded6e7e70def4ae726b41b'

# #add rectangle

colLabels = studyImages[0]['meta']['colLabels'].split(",")
rowLabels = studyImages[0]['meta']['rowLabels'].split(",")

# tileInfo = gc.get('item/%s/tiles' % imgId)
# #https://styx.neurology.emory.edu/girder/api/v1/item/62bb03e4e70def4ae72689f5/tiles/region?units=base_pixels&magnification=0.625&exact=false&encoding=JPEG&jpegQuality=95&jpegSubsampling=0


In [ ]:
# tmaWidth_s,tmaHeight_s, theta_s,leftX_s,leftY_s = tma.generateTMAcontrols(rowLabels,colLabels, region_im)




In [6]:
## So I can use the first slide in the folder to prepopulate this..

# ## Convert image to a numpy array so I can plot it with matplotlib

tmaCore = region_im 
### can use the region_im.shape to set sensible defaults for the tmaWidth and Height
tmaWidth_s,tmaHeight_s, theta_s,leftX_s,leftY_s = tma.generateTMAcontrols(rowLabels,colLabels, region_im)

annotationElements = []



imgOptions = [(x['name'],x['_id']) for x in studyImages]
imgList_dd = widgets.Dropdown(
    options=imgOptions,
#     value=2,
    description='Image:',
)

# def dropdown_eventhandler(change):
#     pass
#     ##print(change.new)

# imgList_dd.observe(dropdown_eventhandler, names='value')


def on_button_clicked(b):
    with output:
        clear_output()
        print('UPDATING TMA OUTPUT' )
        tma.updateAutoTMA(imgId,annotationElements,gc)

ip = interactive(tma.plotGrid,w=tmaWidth_s,h=tmaHeight_s,x1=leftX_s,y1=leftY_s,theta=theta_s,
                 imgId=imgList_dd,
               annotationElements=fixed(annotationElements),
                gc=fixed(gc),thumbCache=fixed(thumbImageCache))

ctrlLineOne = HBox([tmaWidth_s,tmaHeight_s,theta_s])
ctrlLineTwo = HBox([leftX_s,leftY_s,imgList_dd])

display(VBox([ctrlLineOne,ctrlLineTwo ]))#Show all controls

display(ip.children[-1])#Show the output
#display(ip)#Show the output

button = widgets.Button(description="TMA Updated")
output = widgets.Output()

display(button, output)

button.on_click(on_button_clicked)
#97, 113, -2.60


Output()

Button(description='TMA Updated', style=ButtonStyle())

Output()

In [ ]:
thumbImageCache

In [ ]:
annotationElements

### Rotating a grid...  need to also deal with offset..
X=xcos(θ)+ysin(θ)
Y=−x*sin(θ)+y*cos(θ)


In [ ]:
import os

def export_tma_cores( gc, imgId, tmaAnnotationName='autoTMA'):
    ### This will save cropped regions from the itemID... will pass it the annotation ID as well
    i = gc.get('item/%s' % imgId)
    
    fileRoot = i['meta']['slideNumber']
    
    ## Get autoTMA documents for this item
    tmaDocs=  gc.get('annotation?itemId=%s&name=%s' % (imgId, tmaAnnotationName))
    print(len(tmaDocs),'autoTMA annotations found... will use the most recent one')
    
    ae = gc.get('annotation/%s' % tmaDocs[-1]['_id']) ## Grab the last one... assume that's the most recent.. may need to check
    
    print(len(ae['annotation']['elements']),'total ROIs to export')
    
    roiDir = 'ROI_Export'
    
    if not os.path.isdir(roiDir):
        os.mkdir(roiDir)

    for roi in ae['annotation']['elements']:
        roiFileName = "%s/%s.%s.%d.%d.tiff" % (roiDir,fileRoot,roi['label']['value'],roi['center'][0],roi['center'][1])
        
        if not os.path.isfile(roiFileName):
            print(roiFileName)
            r= tma.get_annotation_region(gc,imgId,roi)        

            with open(roiFileName,"wb") as fpi:
                fpi.write(r)
        
        
    #print(i,fileRoot)
export_tma_cores(gc,'62ded6e7e70def4ae726b41b')


In [ ]:
tmaCore = region_im[ys:ye,xs:xe,:]

img=np.copy(tmaCore)
#ret,th2 = cv2.threshold(~img,20,255,cv2.THRESH_BINARY)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (7, 7), 0)

ret = [0,0,0,0]  ## intialize array with 4 values

ret[1],th1 = cv2.threshold(~gray,127,255,cv2.THRESH_BINARY)
(ret[2], th2) = cv2.threshold(blurred, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
ret[3],th3 = cv2.threshold(~img,30,255,cv2.THRESH_BINARY)

### OTSU THRESHOLIND ON BLURRED IMAGE WORKED THE BEST



titles = ['Original Image ', 'Global Thresholding (v = 10) ',
            'Otsu Thresholding ', 'Adaptive Gaussian Thresholding ']
images = [img, th1, th2, th3]
for i in range(4):
    plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i]+str(ret[i]))
    plt.xticks([]),plt.yticks([])
    
    ## Computing Centroid of images as well... will eventually do something with this..
    cX,cY = tma.calculateImgCentroid(images[i],ret[i])
    print('Centroid of image %d is %s %s' % (i,cX,cY))
    
plt.show()

In [ ]:



# plt.rcParams["figure.figsize"] = (15,15)
# plt.imshow(region_im)

# tmaWidth = ( bottomRight[0] - topLeft[0] ) / len(colLabels)
# tmaHeight = ( bottomRight[1] - topLeft[1] ) / len(rowLabels)
# elementScaleFactor = tileInfo['magnification']/dispMag

# cmap = tma.get_cmap( len(colLabels) * len(rowLabels))

# ## While I display the ROI in a matplotlib figure below
# annotationElements = []

# for x,r in enumerate(colLabels):
#     #print("Row: %s" % r)
#     for y,c in enumerate(rowLabels):
#         #print("\t",r,c)

#         x1 = tmaWidth*x+topLeft[0]
#         y1 = tmaHeight*y+topLeft[1]
        
#         plt.gca().add_patch(Rectangle((x1,y1),tmaWidth,tmaHeight,
#                             angle=1,
#                             edgecolor=cmap( random.randint(0,len(colLabels) * len(rowLabels))),
#                             facecolor='none',
#                             lw=2))
        
#         annotationElements.append( {'fillColor': 'rgba(0, 0, 0, 0)',
#     'label': {"value": "%s%s" % (r,c)},
#     'lineColor': tma.generate_random_color(),
#     'lineWidth': 1,
#     'type': 'rectangle',
#     'center': [(x1+tmaWidth/2)*elementScaleFactor,(y1+tmaHeight/2)*elementScaleFactor,0],
#     'width': tmaWidth*elementScaleFactor,
#     'height': tmaHeight*elementScaleFactor,
#     })
        
# #plot fiducial for upper left 
# plt.plot(topLeft[0],topLeft[1], marker='v', color="red")
# #plot fiducial for bottom right
# plt.plot(bottomRight[0],bottomRight[1], marker='x', color="blue")

# #add fiducial for topRight
# plt.plot(topRight[0],topRight[1], marker='o', color="pink")

# tmaAe = {
#   "name": "autoTMA",              # Non-empty string.  Optional
#   "description": "This is a description",  # String.  Optional
#   "elements": annotationElements                           # A list.  Optional.
#                                            # See below for valid elements.
# }
# gc.post('annotation?itemId=%s' % imgId,json=tmaAe)


In [ ]:
# th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
#             cv2.THRESH_BINARY,11,2)
# th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
#             cv2.THRESH_BINARY,11,2)
# convert the image to grayscale and blur it slightly
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# blurred = cv2.GaussianBlur(gray, (7, 7), 0)

# plt.imshow(region_im)


In [ ]:
import cv2 as cv
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.cvtColor(tmaCore, cv2.COLOR_BGR2GRAY)


# global thresholding
ret1,th1 = cv.threshold(img,127,255,cv.THRESH_BINARY)
# Otsu's thresholding
ret2,th2 = cv.threshold(img,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
# Otsu's thresholding after Gaussian filtering
blur = cv.GaussianBlur(img,(5,5),0)
ret3,th3 = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
# plot all the images and their histograms
images = [img, 0, th1,
          img, 0, th2,
          blur, 0, th3]
titles = ['Original Noisy Image','Histogram','Global Thresholding (v=127)',
          'Original Noisy Image','Histogram',"Otsu's Thresholding",
          'Gaussian filtered Image','Histogram',"Otsu's Thresholding"]
for i in range(3):
    plt.subplot(3,3,i*3+1),plt.imshow(images[i*3],'gray')
    plt.title(titles[i*3]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+2),plt.hist(images[i*3].ravel(),256)
    plt.title(titles[i*3+1]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+3),plt.imshow(images[i*3+2],'gray')
    plt.title(titles[i*3+2]), plt.xticks([]), plt.yticks([])
plt.show()